# Explicabilidade do Modelo com SHAP
Neste notebook vamos aplicar SHAP para entender por que o modelo classificou cada transação como baixo, médio ou alto risco.

In [ ]:
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# Carregar a base com score manual
file_path = '/mnt/data/aml_fraud_scored_manual.csv'
df = pd.read_csv(file_path)

In [ ]:
df['manual_risk_score'] = df['manual_risk_score'].map({'low': 0, 'medium': 1, 'high': 2})
df = df.dropna(subset=['manual_risk_score'])

## Selecionar variáveis e treinar modelo novamente

In [ ]:
features = ['amount', 'account_age_days', 'transaction_hour', 'customer_risk_score']
X = df[features]
y = df['manual_risk_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

## Aplicar SHAP

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

## Visualização Global - Importância das variáveis

In [ ]:
shap.summary_plot(shap_values, X_test)

## Visualização Individual - Por que uma transação foi classificada como risco alto, médio ou baixo?

In [ ]:
# Escolher um exemplo específico
sample = X_test.iloc[0:1]
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][0], sample)